<a href="https://colab.research.google.com/github/nicoavilan/Semillero-en-Computacion-Cuantica/blob/main/Remote_quantum_computer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guía para usar un **Computador Cuántico Real** de *manera remota*

La api_key se toma de la página [IBM Quantum Platform](https://quantum.ibm.com/) habiéndome registrado con mi cuenta personal.

In [19]:
#api_key = "Pega acá el API TOKEN"
api_key = " "

* *qiskit*: La biblioteca principal de Qiskit, que proporciona herramientas para crear, manipular y ejecutar circuitos cuánticos. Incluye módulos para definir qubits, puertas cuánticas y simulaciones básicas.
* *qiskit-nature*: Una biblioteca de Qiskit diseñada para aplicaciones en ciencias naturales, como química cuántica y simulaciones de sistemas físicos (por ejemplo, moléculas o materiales). Se usa para resolver problemas como la energía fundamental de moléculas.
* *qiskit-aer*: Proporciona simuladores cuánticos de alto rendimiento. Incluye simuladores como AerSimulator, que permite simular circuitos cuánticos en una computadora clásica, con o sin ruido, para probar algoritmos antes de ejecutarlos en hardware real.
* *qiskit-algorithms*: Contiene implementaciones de algoritmos cuánticos populares, como el algoritmo de Shor, Grover, o el Variational Quantum Eigensolver (VQE). Es útil para tareas de optimización, aprendizaje automático cuántico, y más.
* *qiskit-ibm-runtime*: Permite interactuar con los servicios en la nube de IBM Quantum, como computadores cuánticos reales y simuladores avanzados. Proporciona herramientas como Sampler y Estimator para ejecutar circuitos de manera eficiente en hardware real o simuladores remotos.

In [ ]:
%pip install qiskit --quiet

In [ ]:
%pip install qiskit-aer --quiet

In [ ]:
%pip install qiskit-algorithms --quiet

El siguiente código genera un mensaje de error por conflictos por incompatibilidades, sin embargo funciona. Habrá que probar nuevamente cuando todos los módulos estén actualizados a la nueva versión.  

In [ ]:
%pip install qiskit-ibm-runtime --quiet

`pylatexenc` es una biblioteca de Python diseñada para generar y manipular código LaTeX de manera programática.

In [ ]:
%pip install pylatexenc --quiet

Verifico la versión. Este código funciona en versión 2.1.0

In [ ]:
from qiskit import __version__
print(__version__)

In [9]:
from qiskit import QuantumCircuit#, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Statevector # Pauli, SparsePauliOp

In [ ]:
qc = QuantumCircuit(2,2)
qc.draw(output="mpl")

In [ ]:
init_state = Statevector(qc)
display(init_state.draw("latex"))
print("\n")
plot_bloch_multivector(init_state)

Combina el uso de una compuerta Hadamard y una CNOT para generar un estado de Bell

In [ ]:
qc.h(0)
qc.cx(0, 1)
qc.draw("mpl")

In [ ]:
display(Statevector(qc).draw("latex"))

In [ ]:
h_state = Statevector(qc)
display(h_state.draw("latex"))

Reconocemos el estado de Bell ([Bell state](https://en.wikipedia.org/wiki/Bell_state))

Medimos

In [ ]:
qc.measure(0,0)
qc.measure(1,1)
qc.draw("mpl")

In [ ]:
backend = AerSimulator()
result = backend.run(qc, shots = 1024).result()
counts = result.get_counts()
print(counts)
plot_histogram(counts)

## Conecto con el computador real

In [17]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler # Session

In [21]:
service = QiskitRuntimeService(channel="local", token=api_key)

Otros posibles canales son:
* "ibm_cloud"
* "ibm_quantum_platform"
* "local"

Estos métodos se están actualizando (junio de 2025) si algo falla hay que revisar la guía de migración de la plataforma:

[Migration guides](https://quantum.cloud.ibm.com/docs/es/migration-guides/classic-iqp-to-cloud-iqp)


Podemos averiguar sobre los posibles servidores con `service.backends()`

In [ ]:
service.backends()

Conecta con un compuador cuántico real como estos:

* IBMBackend('ibm_brisbane')
* BMBackend('ibm_kyiv') [until 18 April 2025]
* IBMBackend('ibm_sherbrooke')

[Upcoming QPU retirements](https://docs.quantum.ibm.com/announcements/product-updates/2025-02-20-retirements-kyiv-sherbrooke)

In [ ]:
backend_qc = service.least_busy()
print(backend_qc)

In [24]:
pm = generate_preset_pass_manager(optimization_level =1, backend = backend_qc)
transpiled_circuit = pm.run(qc)

In [25]:
sampler = Sampler(mode=backend_qc)

In [26]:
job = sampler.run([transpiled_circuit])

El método `.status()` indica el estado del proceso:

[JobStatus](https://docs.quantum.ibm.com/api/qiskit/qiskit.providers.JobStatus)

 * INITIALIZING
 * QUEUED
 * VALIDATING
 * RUNNING
 * CANCELLED
 * DONE
 * ERROR

In [ ]:
job.status()

In [48]:
result = job.result()

In [ ]:
display(result)

In [ ]:
qc.cregs

In [ ]:
result[0].data['c'].get_counts()

In [ ]:
counts = result[0].data['c'].get_counts()
print(counts)
plot_histogram(counts)

[Nicolás Avilán Vargas](http://www.linkedin.com/in/nicoavilanv)

Para reportar errores o sugerencias: nicolasg.avilan@urosario.edu.co